In [50]:
import pandas as pd
import numpy as np
from  bs4 import BeautifulSoup
import requests

In [51]:
#일단 현대 차만 해보자
url = "https://auto.naver.com/company/main.nhn?mnfcoNo=16&importYn=N&page=1"
res = requests.get(url)
soup = BeautifulSoup(res.text,"html.parser")

In [52]:
page_url_list = soup.find("div",{"class":"paginate2"}).find_all("a")
page_url_list = ["https://auto.naver.com"+ x['href'] for x in page_url_list]
page_url_list  = list(set(page_url_list))
page_url_list.append(url)
page_url_list.reverse()
car_url_list = []

In [53]:
url = "https://auto.naver.com/company/main.nhn?mnfcoNo=16&modelType=DC&order=0&importYn=&page=1"

while(1):
    page_url_list.append(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text,"html.parser")

    temp = soup.find("div",{"class":"paginate2"}).select("a.next")
    temp = ["https://auto.naver.com"+ x['href'] for x in temp]
    if not temp:
        break
    url=str(temp[0])
    

In [54]:
for x in page_url_list :
    res = requests.get(x)
    soup = BeautifulSoup(res.text,"html.parser")

    temp_list = soup.find_all("a",{"class":"model_name"})
    [ car_url_list.append("https://auto.naver.com/"+ x['href']) for x in temp_list]

In [55]:
# 자동차 종류 별로 DF 를 만들어 보자
org_col = ["차 이름","연식","모델명","가격","주요제원[엔진형식]","주요제원[과급방식]","주요제원[배기량]","주요제원[연료]","주요제원[연비(등급)]","주요제원[승차인원]","주요제원[구동방식]","주요제원[변속기]", #11
          "엔진[엔진형식]","엔진[배기량]","엔진[연료]","엔진[최대출력]","엔진[최대토크]",
          "성능[최고속도]","성능[가속성능(0~100km/h)]","성능[제동성능]","성능[연비(등급)]","성능[CO²배출량]", #5
          "치수[전장]","치수[전폭]","치수[전고]","치수[축거]","치수[윤거전]","치수[윤거후]","치수[공차중량]","치수[승차인원]","치수[전륜타이어]","치수[후륜타이어]", #10
          "섀시[구동방식]","섀시[변속기]","섀시[전륜서스펜션]","섀시[후륜서스펜션]","섀시[전륜제동장치]","섀시[후륜제동장치]","섀시[스티어링]", "Error1","Error2"] # 7
hybrid_col = ["차 이름","연식","모델명","가격","주요제원[엔진형식]","주요제원[과급방식]","주요제원[배기량]","주요제원[연료]","주요제원[연비(등급)]","주요제원[승차인원]","주요제원[구동방식]","주요제원[변속기]", #11
          "엔진[엔진형식]","엔진[배기량]","엔진[연료]","엔진[최대출력]","엔진[최대토크]","엔진[합산출력]","엔진[합산토크]", # 7
          "성능[최고속도]","성능[가속성능(0~100km/h)]","성능[제동성능]","성능[연비(등급)]","성능[CO²배출량]", # 5
          "치수[전장]","치수[전폭]","치수[전고]","치수[축거]","치수[윤거전]","치수[윤거후]","치수[공차중량]","치수[승차인원]","치수[전륜타이어]","치수[후륜타이어]",  #10
          "섀시[구동방식]","섀시[변속기]","섀시[전륜서스펜션]","섀시[후륜서스펜션]","섀시[전륜제동장치]","섀시[후륜제동장치]","섀시[스티어링]"] # 7
hydrogen_col = ["차 이름","연식","모델명","가격","주요제원[엔진형식]","주요제원[과급방식]","주요제원[배기량]","주요제원[연료]","주요제원[연비(등급)]","주요제원[승차인원]","주요제원[구동방식]","주요제원[변속기]",
          "엔진[엔진형식]","엔진[배기량]","엔진[연료]","엔진[항속거리]","엔진[수소연료탱크용량]","엔진[모터최대출력]","엔진[모터최대토크]", # 7
          "성능[최고속도]","성능[가속성능(0~100km/h)]","성능[제동성능]","성능[연비(등급)]","성능[CO²배출량]",
          "치수[전장]","치수[전폭]","치수[전고]","치수[축거]","치수[윤거전]","치수[윤거후]","치수[공차중량]","치수[승차인원]","치수[전륜타이어]","치수[후륜타이어]",
          "섀시[구동방식]","섀시[변속기]","섀시[전륜서스펜션]","섀시[후륜서스펜션]","섀시[전륜제동장치]","섀시[후륜제동장치]","섀시[스티어링]"]
electric_col = ["차 이름","연식","모델명","가격","주요제원[엔진형식]","주요제원[과급방식]","주요제원[배기량]","주요제원[연료]","주요제원[연비(등급)]","주요제원[승차인원]","주요제원[구동방식]","주요제원[변속기]",
          "엔진[엔진형식]","엔진[배기량]","엔진[연료]","엔진[1회 충전 주행가능거리]","엔진[에너지용량]","엔진[모터최대출력]","엔진[모터최대토크]", # 7
          "성능[최고속도]","성능[가속성능(0~100km/h)]","성능[제동성능]","성능[연비(등급)]","성능[CO²배출량]",
          "치수[전장]","치수[전폭]","치수[전고]","치수[축거]","치수[윤거전]","치수[윤거후]","치수[공차중량]","치수[승차인원]","치수[전륜타이어]","치수[후륜타이어]",
          "섀시[구동방식]","섀시[변속기]","섀시[전륜서스펜션]","섀시[후륜서스펜션]","섀시[전륜제동장치]","섀시[후륜제동장치]","섀시[스티어링]"]
org_df = pd.DataFrame(columns=org_col)
hybrid_df = pd.DataFrame(columns=hybrid_col)
hydrogen_df = pd.DataFrame(columns=hydrogen_col)
electric_df = pd.DataFrame(columns=electric_col)

In [56]:
car_url_list = [x.replace("main","lineup") for x in car_url_list]

In [57]:
# 시판모델
for url in car_url_list:
    res = requests.get(url)
    soup = BeautifulSoup(res.text,"lxml")
    car_name = soup.find("h3").text
    car_info = soup.select("div.col_wrap div.col_contents div.lineup_btm_td")
    car_info = [x.select("li") for x in car_info]
    car_year = car_name.split()[0]
    car_name = car_name.split()[1]
    for i in range(len(car_info)):
        for j in range(len(car_info[i])):
            car_info[i][j] = car_info[i][j].text.replace("\r\n","").replace("(정보없음)","").replace(" ","").replace("\xa0","")
     #제원 항목 과 모델 명 찾기
    # model_name_list , col_name_list, price_list
    model_name_list = soup.select("div.price_section dt")
    model_name_list = [x.text for x in model_name_list ]

    price_list = soup.select("div.price_section dd")
    price_list = [x.text for x in price_list ]
    for i in range(len(price_list)):
        car_info[i].insert(0,price_list[i])
        car_info[i].insert(0,model_name_list[i])
        car_info[i].insert(0,car_year)
        car_info[i].insert(0,car_name)
        
        if len(car_info[i])==39:
            car_info[i].append("")
            car_info[i].append("")
            
        print(car_name)
        if car_info[i][7]=="가솔린" or car_info[i][7]=="LPG" or car_info[i][7]=="디젤" :
            org_df = org_df.append(pd.Series(car_info[i],index=org_col),ignore_index=True)
        elif car_info[i][7]=="전기" :
            electric_df = electric_df.append(pd.Series(car_info[i],index=electric_col),ignore_index=True)
        elif car_info[i][7] =="수소" :
            hydrogen_df = hydrogen_df.append(pd.Series(car_info[i],index=hydrogen_col),ignore_index=True)
        else :
            hybrid_df = hybrid_df.append(pd.Series(car_info[i],index=hybrid_col),ignore_index=True)

투싼
투싼
투싼
투싼
아반떼
아반떼
아반떼
아반떼
싼타페
싼타페
팰리세이드
팰리세이드
팰리세이드
팰리세이드
그랜저
그랜저
그랜저
그랜저
코나
코나
베뉴
아반떼
투싼
쏘나타
쏘나타
쏘나타
쏘나타
그랜저
쏘나타
그랜드
그랜드
그랜드
그랜드
그랜드
그랜드
그랜드
코나
넥쏘
아이오닉
i30
i30
i30
i30
벨로스터
아이오닉
코나
포터2
포터2
포터2
포터2
포터2
포터2
쏠라티
쏠라티
i30
i30
벨로스터
벨로스터
포터2
아이오닉
라페스타
그랜드
크레타
크레타
i10
i10
i10
i20
i20
i30
i30
투싼
투싼
투싼
투싼
투싼
투싼
코나
코나
코나
코나
싼타페
싼타페
싼타페
싼타페
싼타페
싼타페
싼타페
싼타페
싼타페
싼타페
싼타페
싼타페
아반떼
아반떼
아반떼
아반떼
아반떼
투싼
투싼
투싼
그랜저
그랜저
그랜저
그랜저
그랜저
쏘나타
쏘나타
쏘나타
쏘나타
아반떼
아반떼
아반떼
아반떼
싼타페
싼타페
싼타페
투싼
투싼
투싼
투싼
투싼
투싼
그랜드
그랜드
그랜저
그랜저
그랜저
투싼
투싼
투싼
투싼
투싼
투싼
투싼
쏘나타
쏘나타
쏘나타
팰리세이드
팰리세이드
팰리세이드
팰리세이드
쏘나타
쏘나타
쏘나타
쏘나타
아반떼
아반떼
아반떼
아반떼
싼타페
싼타페
싼타페
싼타페
투싼
투싼
투싼
스쿠프
엑센트
그랜저
그랜저
그랜저
그랜저
그랜저
아반떼
아반떼
아반떼
그랜저
그랜저
그랜저
그랜저
그랜저
쏘나타
쏘나타
쏘나타
쏘나타
쏘나타
쏘나타
쏘나타
쏘나타
싼타페
싼타페
싼타페
투싼
투싼
투싼
싼타페
싼타페
싼타페
싼타페
싼타페
그랜저
그랜저
그랜저
코나
코나
쏘나타
쏘나타
쏘나타
쏘나타
쏘나타
아반떼
아반떼
그랜저
그랜저
그랜저
코나
코나
코나
싼타페
싼타페
싼타페
싼타페
싼타페
싼타페
그랜저
그랜저
그랜저
에쿠스
에쿠스
i40
그랜저
그랜저
그랜저
투싼
투싼
투싼
투싼
투싼
투싼
그랜저
그랜저
그랜저
그랜저
그랜저
아반떼
아반떼
아반떼
그랜저
그랜저
그랜저
그랜저
맥스크루즈
맥스크루즈
투싼
투싼
투싼
싼타페
싼타페
싼타페
싼타페
투싼
투싼
투싼
투싼
투싼
투싼
투스카니
투스카

In [58]:

# df = pd.concat([df,org_df],ignore_index=True,axis=0)
org_df = pd.concat([org_df,hybrid_df],ignore_index=True,axis=0)
org_df = pd.concat([org_df,hydrogen_df],ignore_index=True,axis=0)
org_df = pd.concat([org_df,electric_df],ignore_index=True,axis=0)
# df = df.append(hybrid_df)
# df = df.append(hydrogen_df)
# df = df.append(electric_df)

org_df

# df.to_csv("total_car.csv",encoding="utf-8-sig",index=False,sep=",")

,차 이름,연식,모델명,가격,주요제원[엔진형식],주요제원[과급방식],주요제원[배기량],주요제원[연료],주요제원[연비(등급)],주요제원[승차인원],...,Error1,Error2,엔진[합산출력],엔진[합산토크],엔진[항속거리],엔진[수소연료탱크용량],엔진[모터최대출력],엔진[모터최대토크],엔진[1회 충전 주행가능거리],엔진[에너지용량]
0,투싼,2021,1.6 가솔린 터보,"2,435~3,155만원",I4,싱글터보,"1,598cc",가솔린,DCT12.0~12.5km/ℓ(3등급),5인승,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,투싼,2021,2.0 디젤,"2,626~3,346만원",I4,싱글터보,"1,998cc",디젤,자동13.9~14.8km/ℓ(2등급),5인승,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,투싼,2021,1.6 가솔린 터보...,"2,631~3,351만원",I4,싱글터보,"1,598cc",가솔린,DCT11.0~11.6km/ℓ(3등급~4등급),5인승,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,투싼,2021,2.0 디젤 AWD,"2,847~3,567만원",I4,싱글터보,"1,998cc",디젤,자동13.0~13.3km/ℓ(3등급),5인승,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,아반떼,2020,1.6 가솔린,"1,570~2,453만원",I4,자연흡기,"1,598cc",가솔린,수동14.4km/ℓ(2등급)CVT14.9~15.4km/ℓ(2등급),5인승,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,아이오닉,2017,I,"3,840만원",,정보없음,정보없음,전기,자동6.3km/kWh,5인승,...,NaN,NaN,NaN,NaN,NaN,NaN,88.0Kw,295.0Nm,191km,28.0kWh
943,아이오닉,2017,N,"4,000만원",,정보없음,정보없음,전기,자동6.3km/kWh,5인승,...,NaN,NaN,NaN,NaN,NaN,NaN,88.0Kw,295.0Nm,191km,28.0kWh
944,아이오닉,2017,Q,"4,300만원",,정보없음,정보없음,전기,자동6.3km/kWh,5인승,...,NaN,NaN,NaN,NaN,NaN,NaN,88.0Kw,295.0Nm,191km,28.0kWh
945,아이오닉,2016,n,"4,000만원",,정보없음,정보없음,전기,자동6.3km/kWh,5인승,...,NaN,NaN,NaN,NaN,NaN,NaN,88.0Kw,295.0Nm,191km,28.0kWh


In [61]:
# #df sort
org_df=org_df.sort_values(by=["차 이름","연식"])
# hybrid_df=hybrid_df.sort_values(by=["차 이름","연식"])
# hydrogen_df=hydrogen_df.sort_values(by=["차 이름","연식"])
# electric_df=electric_df.sort_values(by=["차 이름","연식"])



In [62]:
#df to csv 
org_df.to_csv("org_car.csv",encoding="utf-8-sig",index=False,sep=",")
hybrid_df.to_csv("hybrid_car.csv",encoding="utf-8-sig",index=False,sep=",")
hydrogen_df.to_csv("hydrogen_car.csv",encoding="utf-8-sig",index=False,sep=",")
electric_df.to_csv("electric_car.csv",encoding="utf-8-sig",index=False,sep=",")